# 🧱 `DIM_CURSO` (Gold) — Notebook pronto (lendo Silver e salvando em Gold)

Este notebook foi preparado para a estrutura:

```
/bronze
/silver
/gold
```

✅ Ele:
- lê **2018_anonimizado.xlsx** e **2019_anonimizado.xlsx** a partir de `../silver/`
- consolida em um único dataframe
- cria a dimensão **DIM_CURSO**
- preenche `AREA_GERAL_CINE` vazia como **"Não informado"**
- deduplica por `id_curso` mantendo o registro mais completo
- salva em `gold/output/dim_curso.csv` (ou seja, na mesma pasta do notebook, dentro de `output/`)

> **Como usar:** coloque este notebook dentro da pasta `gold/` do seu repositório e rode as células em ordem.


## 0) Imports e configuração de display

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)


## 1) Paths robustos (funciona com o notebook dentro de `gold/`)

In [ ]:
# ----------------------------------------------------
# NOTEBOOK deve estar em /gold
# Entradas em ../silver
# Saída em ./output
# ----------------------------------------------------
BASE_DIR = Path().resolve()  # pasta atual (gold/)

INPUT_FILES = [
    (BASE_DIR / ".." / "silver" / "2018_anonimizado.xlsx").resolve(),
    (BASE_DIR / ".." / "silver" / "2019_anonimizado.xlsx").resolve(),
]

OUT_DIR = (BASE_DIR / "output").resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_DIM_CURSO = OUT_DIR / "dim_curso.csv"

print("📌 BASE_DIR:", BASE_DIR)
print("📥 INPUT_FILES:")
for f in INPUT_FILES:
    print(" -", f)
print("📤 OUT_DIM_CURSO:", OUT_DIM_CURSO)


## 2) Leitura dos arquivos (Silver) e consolidação

In [ ]:
dfs = []
for f in INPUT_FILES:
    if not f.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {f}")
    tmp = pd.read_excel(f, dtype=str)
    tmp["fonte_arquivo"] = f.name  # opcional (debug)
    dfs.append(tmp)

df = pd.concat(dfs, ignore_index=True)

print("✅ Arquivos lidos:", [f.name for f in INPUT_FILES])
print("✅ Linhas/Colunas consolidadas:", df.shape)
df.head()


## 3) Funções auxiliares (padronização e deduplicação)

In [ ]:
def norm_missing(s: pd.Series) -> pd.Series:
    """Padroniza nulos e strings vazias."""
    x = s.astype(str).str.strip()
    x = x.replace({
        "": np.nan,
        "nan": np.nan, "NAN": np.nan,
        "None": np.nan, "NONE": np.nan
    })
    return x

def dedup_most_complete(df_in: pd.DataFrame, key: str) -> pd.DataFrame:
    """Mantém a linha mais 'completa' por chave (mais colunas preenchidas)."""
    score = df_in.notna().sum(axis=1)
    return (df_in.assign(_score=score)
              .sort_values([key, "_score"], ascending=[True, False])
              .drop_duplicates(subset=[key], keep="first")
              .drop(columns=["_score"]))


## 4) Validação de colunas do layout atual

In [ ]:
required = [
    "CODIGO_DO_CURSO",
    "NOME_CURSO_REGULACAO",      # se quiser usar CURSO_CADASTRO, ajuste aqui e na célula seguinte
    "GRAU",
    "CARGA_HORARIA_CADASTRO",
    "CODIGO_AREA_GERAL_CINE",
    "AREA_GERAL_CINE",
]

missing = [c for c in required if c not in df.columns]
if missing:
    raise KeyError(f"Colunas não encontradas no dataframe: {missing}")

print("✅ Colunas obrigatórias OK.")


## 5) Construção da dimensão `DIM_CURSO`

In [ ]:
dim_curso = df[[
    "CODIGO_DO_CURSO",
    "NOME_CURSO_REGULACAO",
    "GRAU",
    "CARGA_HORARIA_CADASTRO",
    "CODIGO_AREA_GERAL_CINE",
    "AREA_GERAL_CINE",
]].copy()

# Padronização
dim_curso["CODIGO_DO_CURSO"] = norm_missing(dim_curso["CODIGO_DO_CURSO"])
dim_curso["NOME_CURSO_REGULACAO"] = norm_missing(dim_curso["NOME_CURSO_REGULACAO"])
dim_curso["GRAU"] = norm_missing(dim_curso["GRAU"])
dim_curso["CARGA_HORARIA_CADASTRO"] = norm_missing(dim_curso["CARGA_HORARIA_CADASTRO"])
dim_curso["CODIGO_AREA_GERAL_CINE"] = norm_missing(dim_curso["CODIGO_AREA_GERAL_CINE"])
dim_curso["AREA_GERAL_CINE"] = norm_missing(dim_curso["AREA_GERAL_CINE"]).fillna("Não informado")

# Remover cursos sem id
dim_curso = dim_curso.dropna(subset=["CODIGO_DO_CURSO"])

# Renomear para padrão dimensional
dim_curso = dim_curso.rename(columns={
    "CODIGO_DO_CURSO": "id_curso",
    "NOME_CURSO_REGULACAO": "nome_curso",
    "GRAU": "grau",
    "CARGA_HORARIA_CADASTRO": "carga_horaria_cadastro",
    "CODIGO_AREA_GERAL_CINE": "cod_cine_area_geral",
    "AREA_GERAL_CINE": "cine_area_geral",
})

# Deduplicar por id_curso (mantendo a linha mais completa)
dim_curso = dedup_most_complete(dim_curso, "id_curso")

# Ordenar
dim_curso = dim_curso.sort_values("id_curso").reset_index(drop=True)

print("✅ DIM_CURSO pronta. Linhas:", len(dim_curso))
dim_curso.head(10)


## 6) Exportar CSV (Gold)

In [ ]:
dim_curso.to_csv(OUT_DIM_CURSO, index=False, encoding="utf-8")
print(f"✅ DIM_CURSO salva em: {OUT_DIM_CURSO}")


## 7) Checagens rápidas (opcional)

In [ ]:
print("Top 10 áreas CINE geral:")
display(dim_curso["cine_area_geral"].value_counts().head(10))

print("\n% de nulos por coluna:")
display((dim_curso.isna().mean()*100).round(2).rename("% nulos"))
